In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import time
import local_parameters
import urllib.request 
import os

In [ ]:
cookie = JSESSIONID=''
headers = {
    'User-Agent': local_parameters.UserAgent,
    'Cookie' : local_parameters.Cookie}
session = requests.session()


In [ ]:
%%time

test=False
issueListFilename = "2001-2008URLs.csv"
_source_html_path = 'source_html2'
get_year = 2003
count = 78
first = 1

def save_html(year,source_html): 
    global count, first, my_year
    if first:
        first = 0
        my_year = year
        count += 1
    else:
        if my_year == year:
            count += 1
        else: 
            my_year = year
            count = 1
            
    print(str(year) + '-' + str(count) + '.html')
    f = open(os.path.join(_source_html_path, str(year) + '-' + str(count) + '.html'), 'w')
    f.write(source_html)
    f.close

def get_url_source_html(url):
    response = session.get(str(url), headers=headers)
    redirected_url = response.history[-1].url
    print(redirected_url)
    response = session.get(redirected_url, headers=headers)
    return response.text

def process_url(row):
    print(str(count) + ": " + row.url)
    source_html = get_url_source_html(row.url)
    #time.sleep(5)
    save_html(row.year, source_html)
    #time.sleep(10)

         
if test == False:
    df = pd.read_csv(issueListFilename)
    #df = df[df.year == get_year] 
    for index, row in df.iterrows():
       process_url(row)
    
else:
    #url = 'https://doi.org/10.1093/geront/gnx189'
    year = '0000'
    url = 'https://doi.org/10.1093/geront/43.2.192'
    data = [[year,url]]
    df = pd.DataFrame(data,columns=['year','url'])
    
    process_url(df.url.iloc[0])


In [ ]:
df.reset_index().columns

In [ ]:
#import urllib.request
#d = urllib.request.urlopen("http://www.google.co.uk")
#d.getheader('Set-Cookie')

In [ ]:
def get_url_source_html(url):
    response = session.get(url, headers=headers)
    redirected_url = response.history[-1].url
    print(redirected_url)

    response = session.get(redirected_url, headers=headers)
    return response.text

In [ ]:
def get_soup(myarticle):
    return BeautifulSoup(myarticle, "lxml")

def strip_hyperlinks(mysoup):
    for a in mysoup.findAll('a'):
        a.replaceWith(" %s " % a.string)
    return(mysoup)

def strip_references(mysoup):
    for div in mysoup.find_all("div", {'class':'ref-list'}): 
        div.decompose()
    return(mysoup)

def strip_copywrite(mysoup):
    for div in mysoup.find_all("div", {'class':'copyright copyright-statement'}): 
        div.decompose()
    return(mysoup)

def strip_tables(mysoup):
    codetags = mysoup.find_all('table')
    for codetag in codetags:
       codetag.extract()
    return(mysoup)

def strip_rnt(mysoup):
    return(mysoup.get_text().replace('\r', ' ').replace('\n', ' ').replace('\t', ' '))
 
def clean_article(mysoup):
    #soup = get_soup(myarticle)
    mysoup = strip_hyperlinks(mysoup)
    mysoup = strip_references(mysoup)
    mysoup = strip_copywrite(mysoup)
    mysoup = strip_tables(mysoup)
    soup_text = strip_rnt(mysoup)
    return soup_text


In [ ]:
def parse_source_html(source_html):
    soup = BeautifulSoup(source_html, "html.parser")

    raw_html.append(soup.get_text())
    
    citation_date = soup.find("div", {"class": "citation-date"}).text
    citation_dates.append(citation_date)

    citation = soup.find(attrs={'class': 'ww-citation-primary'}).text
    citations.append(citation)

    article_body = soup.find("div", {"class": "article-body"})
    articles.append(article_body)
    
    types.append(soup.find(attrs={'property': 'og:type'}).attrs['content'])
    titles.append(soup.find(attrs={'property': 'og:title'}).attrs['content'])
    descriptions.append(soup.find(attrs={'property': 'og:description'}).attrs['content'])
    urls.append(soup.find(attrs={'property': 'og:url'}).attrs['content'])
    updated_times.append(soup.find(attrs={'property': 'og:updated_time'}).attrs['content'])
    site_names.append(soup.find(attrs={'property': 'og:site_name'}).attrs['content'])
    
    article = clean_article(article_body)
    clean_articles.append(article)
    

In [ ]:
%%time

test=False
citations = []
citation_dates = []
articles = []
clean_articles = []
types = []
titles = []
descriptions = []
urls = []
updated_times = []
site_names = []
raw_html = []

issueListFilename = "2001-2008URLs.csv"
get_year = 2003
count = 0

def process_url(my_url):
    global count
    print(str(count) + ": " + my_url)
    source_html = get_url_source_html(my_url)
    #time.sleep(5)
    #print(source_html)
    parse_source_html(source_html)
    #time.sleep(10)
    count += 1
         
if test == False:
    df = pd.read_csv(issueListFilename)
    df = df[df.year == get_year] 
    print(len(df.index))
    df.url.apply(lambda x: process_url(x))
    
else:
    #url = 'https://doi.org/10.1093/geront/gnx189'
    url = 'https://doi.org/10.1093/geront/43.2.192'
    print(url)
    source_html = get_url_source_html(url)
    parse_source_html(source_html)


In [ ]:
df = pd.DataFrame({'citation_date': citation_dates,
                    'citation': citations,
                    'title': titles,
                    'type': types,
                    'description': descriptions,
                    'url': urls,
                    'updated_time': updated_times,
                    'site_name': site_names,
                    'article': articles,
                    'clean_article': clean_articles,
                    'raw_html' : raw_html
                    },columns = ['citation_date', 'citation', 'title', 'type', 'description', 'url', 'updated_time', 'site_name', 'article', 'clean_article', 'raw_html'])
print(df.info())
df.head(10)

In [ ]:
df["clean_article"][0]

In [ ]:
count = 0
def my_print(my_x):
    global count
    print(count, len(my_x))
    count += 1

df["clean_article"].apply(lambda x: my_print(x))

In [ ]:
#df['clean_article'].loc[129]
df.loc[129]

In [ ]:
count = 0
remove_indices = []
def get_short_articles(my_x):
    global remove_indices, count
    if len(my_x) < 500:
        remove_indices.append(count)
    count += 1

df["clean_article"].apply(lambda x: get_short_articles(x))
print("Initial set: ", len(df))
print("Remove set: " + str(len(remove_indices)))
print(remove_indices)

df_reduced = df.drop(remove_indices)
print("After removal: " + str(len(df_reduced)))

In [ ]:
count = 0
def my_print(my_x):
    global count
    print(count, len(my_x))
    count += 1

df_reduced["clean_article"].apply(lambda x: my_print(x))

In [ ]:
import os
base_filename = 'articles' + str(get_year) + '.txt'
with open(os.path.join(base_filename),'w') as outfile:
    df_reduced["clean_article"].apply(lambda x: outfile.write(x + '\n'))
outfile.close()
num_lines = sum(1 for line in open(base_filename))
print(num_lines)
            

In [ ]:
import sys
sys.setrecursionlimit(10000)
pickle_file = 'data_' + str(get_year) + '.pkl.gzip'
df_reduced.to_pickle(pickle_file, compression='gzip')

In [ ]:
result_filename = 'articles' + str(get_year) + '.csv'
df_reduced.to_csv(result_filename)